<a href="https://colab.research.google.com/github/linoscjbis80-oss/python/blob/main/%EC%9D%B4%EC%83%81%EC%B9%98%20%ED%83%90%EC%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

\

# Pycaret 패키지 실행

In [114]:
!pip install pycaret

In [115]:
!pip install pycaret[analysis]

In [116]:
!pip install umap-learn
from umap import UMAP

# 필요한 라이브러리 실행

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pycaret의 'mice' 데이터셋 import

In [118]:
from pycaret.datasets import get_data
dataset = get_data('mice')

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,...,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
0,309_1,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,...,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,Control,Memantine,C/S,c-CS-m
1,309_2,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,...,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,Control,Memantine,C/S,c-CS-m
2,309_3,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,...,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,Control,Memantine,C/S,c-CS-m
3,309_4,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,...,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,Control,Memantine,C/S,c-CS-m
4,309_5,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,...,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730,Control,Memantine,C/S,c-CS-m


# 'mice' 데이터셋 정보

실험용 쥐(mouse)에게 약물 처치 후,
뇌 부위에서 단백질 발현 수준을 측정한 데이터

피질의 핵 부분에서 감지 가능한 신호를 생성하는 77개 단백질의 발현 수준

In [119]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MouseID          1080 non-null   object 
 1   DYRK1A_N         1077 non-null   float64
 2   ITSN1_N          1077 non-null   float64
 3   BDNF_N           1077 non-null   float64
 4   NR1_N            1077 non-null   float64
 5   NR2A_N           1077 non-null   float64
 6   pAKT_N           1077 non-null   float64
 7   pBRAF_N          1077 non-null   float64
 8   pCAMKII_N        1077 non-null   float64
 9   pCREB_N          1077 non-null   float64
 10  pELK_N           1077 non-null   float64
 11  pERK_N           1077 non-null   float64
 12  pJNK_N           1077 non-null   float64
 13  PKCA_N           1077 non-null   float64
 14  pMEK_N           1077 non-null   float64
 15  pNR1_N           1077 non-null   float64
 16  pNR2A_N          1077 non-null   float64
 17  pNR2B_N       

# 'mice' 데이터셋 전처리

In [121]:
dataset.isnull().sum().sum()

1396

In [122]:
for col in dataset.columns[1:78]:
  dataset[col].fillna(dataset[col].mean(), inplace=True)

In [123]:
dataset.isnull().sum().sum()

0

# Training dataset과 Test dataset을 95:5 비율로 분리

In [124]:
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (1026, 82)
Unseen Data For Predictions: (54, 82)


In [125]:
data

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,...,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
0,3501_12,0.344930,0.626194,0.383583,2.534561,4.097317,0.303547,0.222829,4.592769,0.239427,...,0.151464,0.455172,0.252700,0.218868,0.249187,1.139493,Ts65Dn,Memantine,S/C,t-SC-m
1,3520_5,0.630001,0.839187,0.357777,2.651229,4.261675,0.253184,0.185257,3.816673,0.204940,...,0.109897,0.496423,0.155008,0.153219,0.205440,1.642886,Control,Memantine,C/S,c-CS-m
2,3414_13,0.555122,0.726229,0.278319,2.097249,2.897553,0.222222,0.174356,1.867880,0.203379,...,0.131053,0.344964,0.136109,0.155530,0.185484,1.657670,Ts65Dn,Memantine,C/S,t-CS-m
3,3488_8,0.275849,0.430764,0.285166,2.265254,3.250091,0.189258,0.157837,2.917611,0.202594,...,0.147373,0.390880,0.127944,0.207671,0.175357,0.893598,Control,Saline,S/C,c-SC-s
4,3501_7,0.304788,0.617299,0.335164,2.638236,4.876609,0.280590,0.199417,4.835421,0.236314,...,0.108966,0.470932,0.245277,0.202171,0.240372,0.795637,Ts65Dn,Memantine,S/C,t-SC-m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,3416_2,0.673085,0.877280,0.341103,2.696291,4.180077,0.194568,0.161127,2.479834,0.222031,...,0.105375,0.402853,0.143983,0.183135,0.205440,1.720543,Ts65Dn,Memantine,C/S,t-CS-m
1022,50810F_6,0.454248,0.616073,0.334571,2.295698,4.393972,0.257261,0.204848,3.968334,0.249181,...,0.103136,0.331336,0.169609,0.183135,0.205440,1.472336,Control,Saline,C/S,c-CS-s
1023,3488_7,0.264036,0.429417,0.276054,2.288072,3.187265,0.188879,0.161256,2.810583,0.192646,...,0.116284,0.392842,0.140689,0.195755,0.199139,0.914582,Control,Saline,S/C,c-SC-s
1024,3429_1,0.853106,1.071848,0.280459,2.080645,3.850307,0.181285,0.154625,2.118502,0.201146,...,0.121565,0.349790,0.105418,0.142835,0.136392,1.653082,Ts65Dn,Memantine,C/S,t-CS-m


In [126]:
data_unseen

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,...,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
0,309_6,0.447506,0.628176,0.367388,2.385939,4.807635,0.218578,0.176233,2.141282,0.195188,...,0.109446,0.439833,0.116657,0.140766,0.142180,1.816389,Control,Memantine,C/S,c-CS-m
1,311_3,0.704633,0.802537,0.350110,2.467733,5.548400,0.205323,0.165058,2.107281,0.171401,...,0.131053,0.454509,0.111089,0.157731,0.158543,1.404481,Control,Memantine,C/S,c-CS-m
2,321_4,0.505093,0.695549,0.376029,2.915585,5.917957,0.226734,0.174271,2.663039,0.190038,...,0.131053,0.591586,0.131515,0.188391,0.205440,1.699260,Control,Memantine,C/S,c-CS-m
3,3415_12,0.429133,0.563175,0.258429,2.028151,3.542553,0.214075,0.176759,3.165139,0.167430,...,0.143068,0.371618,0.118223,0.171071,0.173702,1.405727,Control,Memantine,C/S,c-CS-m
4,3415_15,0.373648,0.471165,0.257909,1.860032,2.938526,0.218262,0.150380,2.610132,0.142571,...,0.149337,0.277665,0.086785,0.126537,0.112690,0.790975,Control,Memantine,C/S,c-CS-m
5,3499_8,0.287307,0.457690,0.264008,2.003859,3.103774,0.185106,0.143939,3.060034,0.159091,...,0.112806,0.405776,0.154614,0.143930,0.156937,1.537163,Control,Memantine,C/S,c-CS-m
6,3520_3,0.755979,1.013383,0.390471,2.972950,4.891420,0.247912,0.190015,3.925399,0.234434,...,0.114674,0.525640,0.149243,0.146475,0.205440,1.626736,Control,Memantine,C/S,c-CS-m
7,3520_13,0.511770,0.678274,0.305297,2.210397,3.029671,0.293281,0.239823,3.000490,0.214321,...,0.150883,0.452327,0.190690,0.186517,0.205440,1.530498,Control,Memantine,C/S,c-CS-m
8,3412_12,0.265617,0.403048,0.321589,2.180160,3.184658,0.274363,0.212144,5.701649,0.223388,...,0.161550,0.479450,0.193256,0.287871,0.282671,0.983728,Control,Memantine,S/C,c-SC-m
9,3419_7,0.234140,0.374374,0.269407,1.885225,2.919240,0.246452,0.190317,4.373122,0.213481,...,0.113425,0.391569,0.230912,0.167090,0.205440,1.105637,Control,Memantine,S/C,c-SC-m


# Pycaret환경 세팅

In [127]:
from pycaret.anomaly import *

exp_ano101 = setup(data, normalize = True,
                   ignore_features = ['MouseID'],
                   session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1026, 82)"
2,Transformed data shape,"(1026, 88)"
3,Ignore features,1
4,Numeric features,77
5,Categorical features,4
6,Preprocess,True
7,Imputation type,simple
8,Numeric imputation,mean
9,Categorical imputation,mode


In [128]:
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pycaret.internal.patches.pyod.CBLOFForceToDouble
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


# 'iforest' 모델 적용

In [129]:
iforest = create_model('iforest')

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [130]:
print(iforest)

IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)


In [131]:
iforest_results = assign_model(iforest)
iforest_results.head()
# Anomaly_Score 가 높으면 결측치 확률 높음

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class,Anomaly,Anomaly_Score
0,0.344930,0.626194,0.383583,2.534561,4.097317,0.303547,0.222829,4.592770,0.239427,1.360164,...,0.252700,0.218868,0.249187,1.139493,Ts65Dn,Memantine,S/C,t-SC-m,0,-0.035529
1,0.630001,0.839187,0.357777,2.651229,4.261675,0.253184,0.185257,3.816673,0.204940,1.716583,...,0.155008,0.153219,0.205440,1.642886,Control,Memantine,C/S,c-CS-m,0,-0.078359
2,0.555122,0.726229,0.278319,2.097249,2.897552,0.222222,0.174356,1.867880,0.203379,1.610137,...,0.136109,0.155530,0.185484,1.657670,Ts65Dn,Memantine,C/S,t-CS-m,0,-0.061515
3,0.275849,0.430764,0.285166,2.265254,3.250091,0.189258,0.157837,2.917610,0.202594,1.734746,...,0.127944,0.207671,0.175357,0.893598,Control,Saline,S/C,c-SC-s,0,-0.074478
4,0.304788,0.617299,0.335164,2.638236,4.876609,0.280590,0.199417,4.835421,0.236314,1.226532,...,0.245277,0.202171,0.240372,0.795637,Ts65Dn,Memantine,S/C,t-SC-m,0,-0.077693


In [132]:
(iforest_results.Anomaly==0).sum()
# 95%

974

In [133]:
(iforest_results.Anomaly==1).sum()
# 5% (이상치)

52

# 시각화 및 다른모델과 비교

In [134]:
plot_model(iforest)
plot_model(iforest, plot = 'umap')

In [135]:
model_knn = create_model('knn')
knn2=assign_model(model_knn)
print((knn2.Anomaly==0).sum())
print((knn2.Anomaly==1).sum())

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

974
52


In [136]:
plot_model(model_knn)
plot_model(model_knn,plot='umap')

In [137]:
model_svm = create_model('svm')
svm2=assign_model(model_svm)
model_knn = create_model('knn')
knn2=assign_model(model_knn)
print((svm2.Anomaly==0).sum())
print((knn2.Anomaly==1).sum())

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

974
52


In [138]:
plot_model(model_svm)
plot_model(model_svm,plot='umap')

# Test dataset 적용

In [140]:
# 5% sample 데이터 test
data_unseen.drop('MouseID', axis=1, inplace=True)
unseen_predictions = predict_model(iforest, data=data_unseen)
pd.DataFrame(unseen_predictions).head()

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,class_t-SC-m,class_c-CS-m,class_t-CS-m,class_c-SC-s,class_c-SC-m,class_t-SC-s,class_c-CS-s,class_t-CS-s,Anomaly,Anomaly_Score
0,0.093970,0.048171,0.972061,0.257351,1.045707,-0.341477,-0.197971,-1.076569,-0.537357,0.033322,...,-0.36907,2.495066,-0.380906,-0.380906,-0.400791,-0.380906,-0.379226,-0.32862,0,-0.078627
1,1.144832,0.746830,0.623041,0.494293,1.846052,-0.660699,-0.611491,-1.102760,-1.265231,1.104273,...,-0.36907,2.495066,-0.380906,-0.380906,-0.400791,-0.380906,-0.379226,-0.32862,0,-0.069881
2,0.329322,0.318132,1.146613,1.791638,2.245332,-0.145059,-0.270585,-0.674675,-0.694944,0.233255,...,-0.36907,2.495066,-0.380906,-0.380906,-0.400791,-0.380906,-0.379226,-0.32862,0,-0.039648
3,0.018878,-0.212285,-1.228977,-0.779095,-0.321125,-0.449910,-0.178506,-0.287922,-1.386731,-0.451389,...,-0.36907,2.495066,-0.380906,-0.380906,-0.400791,-0.380906,-0.379226,-0.32862,0,-0.088104
4,-0.207883,-0.580964,-1.239467,-1.266104,-0.973734,-0.349083,-1.154594,-0.715428,-2.147434,-0.877712,...,-0.36907,2.495066,-0.380906,-0.380906,-0.400791,-0.380906,-0.379226,-0.32862,1,0.015706


In [141]:
unseen_predictions.Anomaly.value_counts()
# 이상치 5%

,count
Anomaly,
0,51
1,3


# 각 모델별 상관관계

In [142]:
# 타겟 변수와의 상관관계만 추출
corr_matrix = iforest_results.corr(numeric_only=True)
target_corr = corr_matrix['Anomaly_Score'].drop(['Anomaly_Score', 'Anomaly'])
pd.DataFrame((target_corr.sort_values(ascending=False)))

,Anomaly_Score
RRP1_N,0.288808
AMPKA_N,0.281376
GFAP_N,0.281304
TIAM1_N,0.273683
pCFOS_N,0.263746
...,...
Ubiquitin_N,-0.133735
pPKCG_N,-0.143186
pGSK3B_Tyr216_N,-0.167196
pCAMKII_N,-0.169907


In [143]:
corr_matrix = knn2.corr(numeric_only=True)
target_corr = corr_matrix['Anomaly_Score'].drop(['Anomaly_Score', 'Anomaly'])
pd.DataFrame((target_corr.sort_values(ascending=False)))

,Anomaly_Score
RRP1_N,0.420489
pELK_N,0.319472
pCFOS_N,0.314953
GFAP_N,0.297867
RAPTOR_N,0.255485
...,...
P70S6_N,-0.142986
pS6_N,-0.145721
ARC_N,-0.145721
nNOS_N,-0.225459


In [144]:
corr_matrix = svm2.corr(numeric_only=True)
target_corr = corr_matrix['Anomaly_Score'].drop(['Anomaly_Score', 'Anomaly'])
pd.DataFrame((target_corr.sort_values(ascending=False)))

,Anomaly_Score
RRP1_N,0.391861
pELK_N,0.352794
BRAF_N,0.322724
GFAP_N,0.317166
pERK_N,0.292207
...,...
Ubiquitin_N,-0.126131
GluR3_N,-0.131936
nNOS_N,-0.190476
pCAMKII_N,-0.203008
